In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Упрощённый слой свёртки
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

# Простая U-Net подобная структура
class SimpleUNet(nn.Module):
    def __init__(self):
        super(SimpleUNet, self).__init__()
        self.encoder = nn.Sequential(
            ConvBlock(3, 64),  # Изменено на 3 канала
            ConvBlock(64, 128)
        )
        self.decoder = nn.Sequential(
            ConvBlock(128, 64),
            ConvBlock(64, 3)  # Изменено на 3 канала
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.decoder(x1)
        return x2

# Простой тренинг луп
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Датасет и лоадер (например CIFAR-10 для RGB изображений)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализация для RGB
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = SimpleUNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучающий цикл
def train_model(num_epochs):
    #model.train()
    for epoch in range(num_epochs):
        for batch_idx, (data, _) in enumerate(train_loader):
            print(data.shape)
            data = data.to(device)
    print(data)

            # Добавляем "шум" к изображениям
            noised_data = data + 0.1 * torch.randn_like(data)

            optimizer.zero_grad()
            output = model(noised_data)
            loss = criterion(output, data)
            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}], Loss: {loss.item():.4f}')
                with torch.no_grad():
                    output_img = output[0].cpu().detach().numpy().transpose(1, 2, 0)  # Изменение порядка для отображения
                    noised_img = noised_data[0].cpu().detach().numpy().transpose(1, 2, 0)
                    original_img = data[0].cpu().detach().numpy().transpose(1, 2, 0)

                    plt.figure(figsize=(9,3))
                    plt.subplot(1, 3, 1)
                    plt.title("Original")
                    plt.imshow((original_img + 1) / 2)  # Обратная нормализация для отображения

                    plt.subplot(1, 3, 2)
                    plt.title("Noised")
                    plt.imshow((noised_img + 1) / 2)  # Обратная нормализация для отображения

                    plt.subplot(1, 3, 3)
                    plt.title("Denoised")
                    plt.imshow((output_img + 1) / 2)  # Обратная нормализация для отображения

                    plt.show()

train_model(5)
